# Retrieving and visualizing COVID-19 data
<font size = "3"> This notebook creates a Dash application that visualizes COVID-19 infection data.

In [1]:
#import libraries
import pandas as pd
import requests
import plotly.express as px
from matplotlib import pyplot as plt
from datetime import datetime
import plotly.io as pio

import dash
import dash_core_components as dcc
import dash_html_components as html

# set pandas options
pd.set_option('display.max_columns', 50)

In [2]:
pio.orca.status

orca status
-----------
    state: unvalidated
    executable: None
    version: None
    port: None
    pid: None
    command: None


In [3]:
# create app variable
app = dash.Dash(__name__)

In [4]:
# retrieve Irish COVID-19 data
url_eire = "https://services1.arcgis.com/eNO7HHeQ3rUcBllm/arcgis/rest/services/CovidStatisticsProfileHPSCIrelandOpenData/FeatureServer/0/query?where=1%3D1&outFields=Date,ConfirmedCovidCases&outSR=4326&f=json"
r_eire = requests.get(url_eire).json()
df_eire = pd.json_normalize(r_eire['features'])
df_eire['Timestamp'] = pd.to_datetime(df_eire['attributes.Date'],unit='ms')
df_eire['Date'] = df_eire['Timestamp'].dt.date
df_eire = df_eire.groupby(['Date'])[['Timestamp','attributes.ConfirmedCovidCases']].max()
latest_max_eire = df_eire.loc[df_eire.index.max()]['attributes.ConfirmedCovidCases']
index_eire = df_eire.index
latest_date_eire = index_eire.max()

In [5]:
# retrieve US COVID-19 data.
url_us = "https://api.covidtracking.com/v1/us/daily.json"
r_us = requests.get(url_us).json()
df_us = pd.DataFrame.from_dict(r_us)

df_us['Datetimes'] = pd.to_datetime(df_us['date'],format='%Y%m%d')
df_us['xtraDates'] = df_us['Datetimes'].dt.date
df_us.set_index('Datetimes',inplace=True,drop=True)
df = df_us[["positiveIncrease","xtraDates"]]

latest_us_date = df_us['xtraDates'].max()
latest_us_max = df_us.loc[df_us.index.max()]["positiveIncrease"]

In [6]:
# create bar chart to visualize Irish COVID-19 data
fig_irl = px.bar(df_eire, x='Timestamp', y='attributes.ConfirmedCovidCases')
fig_irl = fig_irl.update_layout(title="Daily COVID-19 Cases in Ireland",
                        title_font_size=20,
                        xaxis_title="Date",
                        yaxis_title="Daily COVID-19 Cases",
                        paper_bgcolor="black",
                        plot_bgcolor="black",
                        font_color="white")
fig_irl = fig_irl.update_xaxes(tickfont=dict(family='Garamond', color='white', size=18),title_font_size=18)
fig_irl = fig_irl.update_yaxes(tickfont=dict(family='Garamond', color='white', size=16),title_font_size=24)
fig_irl.show("png")

ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    /usr/local/bin
    /usr/bin
    /bin
    /usr/sbin
    /sbin

If you haven't installed orca yet, you can do so using conda as follows:

    $ conda install -c plotly plotly-orca

Alternatively, see other installation methods in the orca project README at
https://github.com/plotly/orca

After installation is complete, no further configuration should be needed.

If you have installed orca, then for some reason plotly.py was unable to
locate it. In this case, set the `plotly.io.orca.config.executable`
property to the full path of your orca executable. For example:

    >>> plotly.io.orca.config.executable = '/path/to/orca'

After updating this executable property, try the export operation again.
If it is successful then you may want to save this configuration so that it
will be applied automatically in fut

In [ ]:
# create bar chart to visualize Irish COVID-19 data
fig_us = px.bar(df_us, x='xtraDates', y='positiveIncrease')
fig_us = fig_us.update_layout(title="Daily COVID-19 Cases in America",
                              title_font_size=20,
                              xaxis_title="Date",
                              yaxis_title="Daily COVID-19 Cases",
                              paper_bgcolor="black",
                              plot_bgcolor="black",
                              font_color="white")
fig_us = fig_us.update_xaxes(tickfont=dict(family='Garamond', color='white', size=18),title_font_size=18)
fig_us = fig_us.update_yaxes(tickfont=dict(family='Garamond', color='white', size=16),title_font_size=24)
fig_us.show('png')

In [ ]:
# create app layout
app.layout = html.Div(style={"backgroundColor":"black"}, children=[
    html.Br(),
    html.H2(f'{latest_date_eire:%B %d, %Y}: {latest_max_eire:,} COVID-19 cases confirmed in Ireland'
            ,style={'color':'white','font-size': '2.25em','font_weight': 'bolder','font-family':'Garamond'
                    ,'textAlign':'center'}),
    html.Br(),
    dcc.Graph(id='ire_cov_graph', figure=fig_irl),
    html.Br(),
    html.H2(f'{latest_us_date:%B %d, %Y}: {latest_us_max:,} COVID-19 cases confirmed in America',
            style={'color':'white','font-size': '2.25em','font_weight': 'bolder','font-family':'Garamond'
                   ,'textAlign':'center'}),
    html.Br(),
    dcc.Graph(id='USA_cov_graph',figure=fig_us),
    ]
)
# run app
if __name__ == '__main__':
    app.run_server(debug=False)